In [23]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [24]:
def get_sp500_data(csv_path: str,
                   years: int = 10,
                   end_date: str = None) -> pd.DataFrame:
    """
    Download S&P 500 (ticker ^GSPC) daily data for the last `years` years,
    filter to date, open and close, save to csv, and return DataFrame.
    
    Parameters:
      csv_path: path where the CSV will be saved (and/or read)
      years: how many years back to go
      end_date: end date (string "YYYY-MM-DD"). If None, uses today.
    
    Returns:
      pandas.DataFrame with columns ['date', 'open_price', 'close_price']
    """
    # Determine end and start date
    if end_date:
        end = pd.to_datetime(end_date)
    else:
        end = pd.to_datetime(datetime.today().date())
    start = end - pd.DateOffset(years=years)
    
    # Use yfinance to download data
    ticker = "^GSPC"
    df = yf.download(tickers=ticker,
                     start=start.strftime("%Y-%m-%d"),
                     end=(end + pd.DateOffset(days=1)).strftime("%Y-%m-%d"),
                     interval="1d",
                     auto_adjust=False,   # use raw prices (not split/div adj)
                     progress=False)
    
    # The DataFrame index should be date, and there will be columns:
    #   Open, High, Low, Close, Adj Close, Volume
    # We pick only Open and Close.
    df = df.reset_index()  # bring date from index to column
    df = df.rename(columns={
        'Date': 'date',
        'Open': 'open_price',
        'Close': 'close_price'
    })
    df = df[['date', 'open_price', 'close_price']]
    
    # Sort by date ascending (just in case)
    df = df.sort_values('date').reset_index(drop=True)
    
    # Filter to ensure date >= start
    df = df[df['date'] >= start].copy()
    
    # Save to CSV
    df.to_csv(csv_path, index=False)
    
    return df


In [25]:
csv_out = "sp500_last10years.csv"
df_sp500 = get_sp500_data(csv_out, years=10)
print(df_sp500.head())
print(df_sp500.tail())
print(f"Saved {len(df_sp500)} rows to {csv_out}")


Price        date   open_price  close_price
Ticker                   ^GSPC        ^GSPC
0      2015-11-25  2089.300049  2088.870117
1      2015-11-27  2088.820068  2090.110107
2      2015-11-30  2090.949951  2080.409912
3      2015-12-01  2082.929932  2102.629883
4      2015-12-02  2101.709961  2079.510010
Price        date   open_price  close_price
Ticker                   ^GSPC        ^GSPC
2510   2025-11-19  6625.839844  6642.160156
2511   2025-11-20  6737.930176  6538.759766
2512   2025-11-21  6555.770020  6602.990234
2513   2025-11-24  6636.540039  6705.120117
2514   2025-11-25  6697.029785  6767.160156
Saved 2515 rows to sp500_last10years.csv


In [26]:
# !pip install plotly

In [27]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Fix MultiIndex columns if they exist
if isinstance(df_sp500.columns, pd.MultiIndex):
    df_sp500.columns = df_sp500.columns.get_level_values(0)
    print("Flattened MultiIndex columns to:", df_sp500.columns.tolist())

# Ensure date column is datetime
df_sp500['date'] = pd.to_datetime(df_sp500['date'])

# Remove any rows with invalid dates or prices
df_sp500 = df_sp500.dropna(subset=['date', 'close_price'])
df_sp500 = df_sp500.sort_values('date').reset_index(drop=True)

# Debug: Check data
print(f"Data shape: {df_sp500.shape}")
print(f"Date range: {df_sp500['date'].min()} to {df_sp500['date'].max()}")
print(f"Close price range: {df_sp500['close_price'].min():.2f} to {df_sp500['close_price'].max():.2f}")

# ---- Time windows ----
end = df_sp500['date'].max()
one_year = df_sp500[df_sp500['date'] >= end - pd.DateOffset(years=1)].copy()
five_year = df_sp500[df_sp500['date'] >= end - pd.DateOffset(years=5)].copy()
ten_year = df_sp500.copy()

# Debug: Check filtered data
print(f"\n1-year data points: {len(one_year)}")
print(f"5-year data points: {len(five_year)}")
print(f"10-year data points: {len(ten_year)}")

# ---- Create interactive Plotly subplots ----
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=False,
    subplot_titles=("1-Year Trend", "5-Year Trend", "10-Year Trend")
)

# 1-Year
fig.add_trace(
    go.Scatter(x=one_year['date'], y=one_year['close_price'], mode='lines', name="1 Year"),
    row=1, col=1
)


# 5-Year
fig.add_trace(
    go.Scatter(x=five_year['date'], y=five_year['close_price'], mode='lines', name="5 Year"),
    row=2, col=1
)

# 10-Year
fig.add_trace(
    go.Scatter(x=ten_year['date'], y=ten_year['close_price'], mode='lines', name="10 Year"),
    row=3, col=1
)


fig.update_layout(
    height=900,
    title="S&P 500 Close Price Trends (Log Scale)",
    showlegend=False
)

# Update y-axes to log scale
fig.update_yaxes(type="log", title_text="Close Price ($)", row=1, col=1)
fig.update_yaxes(type="log", title_text="Close Price ($)", row=2, col=1)
fig.update_yaxes(type="log", title_text="Close Price ($)", row=3, col=1)

# Update x-axis label for bottom subplot
fig.update_xaxes(title_text="Date", row=3, col=1)

fig.show()

Flattened MultiIndex columns to: ['date', 'open_price', 'close_price']
Data shape: (2515, 3)
Date range: 2015-11-25 00:00:00 to 2025-11-25 00:00:00
Close price range: 1829.08 to 6890.89

1-year data points: 251
5-year data points: 1256
10-year data points: 2515


In [28]:
def add_bollinger_bands(df, window=20):
    df['sma'] = df['close_price'].rolling(window).mean()
    df['std'] = df['close_price'].rolling(window).std()
    df['upper'] = df['sma'] + 2 * df['std']
    df['lower'] = df['sma'] - 2 * df['std']
    return df

In [29]:
one_year = add_bollinger_bands(one_year)
five_year = add_bollinger_bands(five_year)
ten_year = add_bollinger_bands(ten_year)


In [31]:
fig_bollinger = make_subplots(
    rows=3, cols=1,
    shared_xaxes=False,
    subplot_titles=("1-Year Bollinger", "5-Year Bollinger", "10-Year Bollinger")
)


# SMA
fig_bollinger.add_trace(
    go.Scatter(x=one_year['date'], y=one_year['sma'], mode='lines', name="1Y SMA", line=dict(width=1, dash="dot")),
    row=1, col=1
)

# Upper band
fig_bollinger.add_trace(
    go.Scatter(x=one_year['date'], y=one_year['upper'], mode='lines', name="1Y Upper Band", line=dict(width=1)),
    row=1, col=1
)

# Lower band
fig_bollinger.add_trace(
    go.Scatter(x=one_year['date'], y=one_year['lower'], mode='lines', name="1Y Lower Band", line=dict(width=1)),
    row=1, col=1
)

#5 year bollinger
fig_bollinger.add_trace(go.Scatter(x=five_year['date'], y=five_year['sma'], mode='lines', name="5Y SMA", line=dict(width=1, dash="dot")),
              row=2, col=1)
fig_bollinger.add_trace(go.Scatter(x=five_year['date'], y=five_year['upper'], mode='lines', name="5Y Upper", line=dict(width=1)),
              row=2, col=1)
fig_bollinger.add_trace(go.Scatter(x=five_year['date'], y=five_year['lower'], mode='lines', name="5Y Lower", line=dict(width=1)),
              row=2, col=1)

#10 year bollinger
fig_bollinger.add_trace(go.Scatter(x=ten_year['date'], y=ten_year['sma'], mode='lines', name="10Y SMA", line=dict(width=1, dash="dot")),
              row=3, col=1)
fig_bollinger.add_trace(go.Scatter(x=ten_year['date'], y=ten_year['upper'], mode='lines', name="10Y Upper", line=dict(width=1)),
              row=3, col=1)
fig_bollinger.add_trace(go.Scatter(x=ten_year['date'], y=ten_year['lower'], mode='lines', name="10Y Lower", line=dict(width=1)),
              row=3, col=1)


fig_bollinger.update_layout(
    height=900,
    title="S&P 500 Bollinger Bands Close Price Trends (Log Scale)",
    showlegend=False
)

# Update y-axes to log scale
fig_bollinger.update_yaxes(type="log", title_text="Close Price ($)", row=1, col=1)
fig_bollinger.update_yaxes(type="log", title_text="Close Price ($)", row=2, col=1)
fig_bollinger.update_yaxes(type="log", title_text="Close Price ($)", row=3, col=1)

# Update x-axis label for bottom subplot
fig_bollinger.update_xaxes(title_text="Date", row=3, col=1)

fig_bollinger.show()



In [38]:
# Backtest Bollinger Mean-Reversion (Strategy A)
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
import datetime

# ---------- PARAMETERS ----------
BB_WINDOW = 20
BB_STD_MULT = 2.0
position_size = 1           # shares per trade (changeable)
commission_pct = 0.001      # e.g., 0.001 = 0.1% per trade roundtrip (applied on both entry & exit)
risk_free_rate = 0.0        # for Sharpe (annual)
trading_days_per_year = 252

# ---------- UTILITIES ----------
def compute_bollinger(df, window=BB_WINDOW, std_mult=BB_STD_MULT):
    df = df.copy()
    df['sma'] = df['close_price'].rolling(window).mean()
    df['std'] = df['close_price'].rolling(window).std()
    df['upper'] = df['sma'] + std_mult * df['std']
    df['lower'] = df['sma'] - std_mult * df['std']
    return df

def get_execution_price(df, idx):
    """
    Return execution price for trade executed on row idx (int index).
    Prefer 'open' if present, else 'close'.
    """
    if 'open' in df.columns and not df['open'].isna().iloc[idx]:
        return df['open'].iloc[idx]
    else:
        return df['close_price'].iloc[idx]

# ---------- SIGNAL GENERATION ----------
def generate_signals(df):
    """
    Signal logic:
    - Entry signal (long): close_price < lower band
    - Exit signal: close_price >= sma
    Execution happens next row (i+1).
    Returns df with 'signal' column: 1 = enter (on next bar), -1 = exit (on next bar), 0 = nothing
    """
    df = df.copy().reset_index(drop=True)
    df['below_lower'] = df['close_price'] < df['lower']
    df['at_or_above_sma'] = df['close_price'] >= df['sma']
    df['signal'] = 0

    # Entry when today's close < lower
    df.loc[df['below_lower'], 'signal'] = 1

    # Exit when today's close >= sma
    df.loc[df['at_or_above_sma'], 'signal'] = -1

    # Shift signals to next row for execution (to avoid lookahead)
    df['exec_signal'] = df['signal'].shift(1).fillna(0).astype(int)
    return df

# ---------- BACKTEST ENGINE ----------
def backtest(df):
    """
    Run vectorized-ish backtest.
    Returns a dict with trades, equity series, metrics and df with positions.
    """
    df = df.copy().reset_index(drop=True)
    df = compute_bollinger(df)
    df = generate_signals(df)

    # state
    position = 0                # 0 or 1 (no shorting)
    entry_price = np.nan
    cash = 0.0                  # start at zero capital; equity evolves from P&L (we'll treat initial capital = cost of first entry)
    equity = []
    trades = []

    # Keep track of unrealized P&L for equity curve
    # We'll set initial cash to 0 and compute equity as cumulative pnl + cash.
    cumulative_pnl = 0.0

    # For clarity, store per-row trade executions and position
    df['position'] = 0
    df['trade_price'] = np.nan
    df['trade_type'] = None    # 'entry' or 'exit'
    df['pnl'] = 0.0
    df['cash'] = 0.0
    df['equity'] = 0.0

    for i in range(len(df)):
        row = df.iloc[i]
        exec_sig = int(row['exec_signal'])

        # Execute entry
        if exec_sig == 1 and position == 0:
            # Enter long at next bar price (if exists)
            # Ensure there is an execution bar (we're at bar i)
            exec_price = get_execution_price(df, i)
            position = 1
            entry_price = exec_price
            entry_commission = position_size * exec_price * commission_pct
            cash -= position_size * exec_price + entry_commission
            df.at[i, 'trade_price'] = exec_price
            df.at[i, 'trade_type'] = 'entry'
            trades.append({
                'type': 'entry',
                'index': i,
                'date': df.at[i, 'date'],
                'price': exec_price,
                'size': position_size,
                'commission': entry_commission
            })

        # Execute exit
        # Execute exit
        elif exec_sig == -1 and position == 1:
        # For mean-reverting strategy, exit at SMA price when price crosses back to mean
          exec_price = df.iloc[i]['sma']  # Use SMA price instead of close/open
        # Realized P&L
          gross_pnl = position_size * (exec_price - entry_price)
          exit_commission = position_size * exec_price * commission_pct
          realized = gross_pnl - exit_commission
          cash += position_size * exec_price - exit_commission
          cumulative_pnl += gross_pnl - (position_size * entry_price * commission_pct) - exit_commission
          df.at[i, 'trade_price'] = exec_price
          df.at[i, 'trade_type'] = 'exit'
          df.at[i, 'pnl'] = realized
          trades.append({
          'type': 'exit',
          'index': i,
          'date': df.at[i, 'date'],
          'price': exec_price,  # This will now be the SMA price
          'size': position_size,
          'commission': exit_commission,
          'realized_pnl': realized,
          'entry_price': entry_price
          })
          position = 0
          entry_price = np.nan
        # mark position and equity (mark-to-market)
        df.at[i, 'position'] = position
        # unrealized pnl:
        if position == 1:
          current_price = row['close_price']
          unrealized = position_size * (current_price - entry_price)
        else:
          unrealized = 0.0

        total_equity = cash + unrealized
        df.at[i, 'cash'] = cash
        df.at[i, 'equity'] = total_equity
        equity.append(total_equity)

    # fill forward equity
    df['equity'] = pd.Series(equity, index=df.index)

    # If no trades, return early
    if len(trades) == 0:
        metrics = {'message': 'No trades triggered in this dataset.'}
        return {'df': df, 'trades': trades, 'metrics': metrics}

    # Build per-trade P&L table: match entry-exit pairs
    paired_trades = []
    entry = None
    for t in trades:
        if t['type'] == 'entry':
            entry = t
        elif t['type'] == 'exit' and entry is not None:
            paired = {
                'entry_index': entry['index'],
                'entry_date': entry['date'],
                'entry_price': entry['price'],
                'exit_index': t['index'],
                'exit_date': t['date'],
                'exit_price': t['price'],
                'size': entry['size'],
                'pnl': t['realized_pnl']
            }
            paired_trades.append(paired)
            entry = None

    trades_df = pd.DataFrame(paired_trades)

    # PERFORMANCE METRICS
    # Equity series -> compute returns (daily returns of equity)
    # If initial capital is zero (we used cash based), we want to normalize: set initial capital = first positive cash outflow (cost of first entry) or 1.0
    initial_capital = abs(trades[0]['price'] * position_size) if trades else 1.0
    # Build portfolio value series starting from initial capital (to compute returns)
    portfolio_value = df['equity'].copy()
    # If portfolio starts negative (because of cash negative on first entry), shift to sensible baseline:
    if portfolio_value.iloc[0] <= 0:
        shift = initial_capital - portfolio_value.iloc[0]
        portfolio_value = portfolio_value + shift
    # If portfolio_value contains NaN, fill forward
    portfolio_value = portfolio_value.fillna(method='ffill').fillna(initial_capital)

    # compute daily returns
    pv = portfolio_value
    daily_ret = pv.pct_change().fillna(0)

    total_return = (pv.iloc[-1] / pv.iloc[0]) - 1.0
    num_days = (df['date'].iloc[-1] - df['date'].iloc[0]).days
    years = num_days / 365.25 if num_days > 0 else 1.0
    CAGR = (pv.iloc[-1] / pv.iloc[0]) ** (1.0 / years) - 1.0 if years > 0 else np.nan

    ann_vol = daily_ret.std() * math.sqrt(trading_days_per_year)
    sharpe = (daily_ret.mean() * trading_days_per_year - risk_free_rate) / ann_vol if ann_vol > 0 else np.nan

    # max drawdown
    rolling_max = pv.cummax()
    drawdown = (pv - rolling_max) / rolling_max
    max_drawdown = drawdown.min()

    win_trades = trades_df[trades_df['pnl'] > 0]
    loss_trades = trades_df[trades_df['pnl'] <= 0]
    win_rate = len(win_trades) / len(trades_df) if len(trades_df) else np.nan
    gross_profit = win_trades['pnl'].sum() if len(win_trades) else 0.0
    gross_loss = loss_trades['pnl'].abs().sum() if len(loss_trades) else 0.0
    profit_factor = (gross_profit / gross_loss) if gross_loss > 0 else np.nan

    metrics = {
        'initial_capital_estimate': initial_capital,
        'ending_portfolio_value': pv.iloc[-1],
        'total_return': total_return,
        'CAGR': CAGR,
        'annual_volatility': ann_vol,
        'sharpe': sharpe,
        'max_drawdown': max_drawdown,
        'num_trades': len(trades_df),
        'win_rate': win_rate,
        'profit_factor': profit_factor,
    }

    return {'df': df, 'trades': trades_df, 'metrics': metrics}

# ---------- PLOTTING ----------
def plot_backtest_results(result, title_prefix="Backtest"):
    df = result['df']
    trades = result['trades']
    metrics = result['metrics']

    # Price chart with bands and signals
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        row_heights=[0.65, 0.35],
                        subplot_titles=(f"{title_prefix} Price & Bollinger Signals", "Equity Curve"))

    fig.add_trace(go.Scatter(x=df['date'], y=df['close_price'], mode='lines', name='Close'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['date'], y=df['sma'], mode='lines', name='SMA'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['date'], y=df['upper'], mode='lines', name='Upper'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['date'], y=df['lower'], mode='lines', name='Lower'), row=1, col=1)

    # plot entry/exit markers (use trades df indexes)
    if not trades.empty:
        fig.add_trace(go.Scatter(
            x=trades['entry_date'], y=trades['entry_price'],
            mode='markers', marker=dict(symbol='triangle-up', size=10),
            name='Entry'), row=1, col=1
        )
        fig.add_trace(go.Scatter(
            x=trades['exit_date'], y=trades['exit_price'],
            mode='markers', marker=dict(symbol='triangle-down', size=10),
            name='Exit'), row=1, col=1
        )

    # Equity
    fig.add_trace(go.Scatter(x=df['date'], y=df['equity'], mode='lines', name='Equity'), row=2, col=1)

    fig.update_yaxes(type='log', row=1, col=1)  # keep same log scale if desired
    fig.update_layout(height=900, title_text=f"{title_prefix} (BB Mean Reversion)")

    fig.show()

    # Print metrics
    print("=== Performance Metrics ===")
    for k, v in metrics.items():
        if isinstance(v, float):
            print(f"{k}: {v:.6f}")
        else:
            print(f"{k}: {v}")

    # Show trades table if any
    if not trades.empty:
        display(trades)

# ---------- RUN FOR EACH TIME HORIZON ----------
def run_all_backtests(df_sp500):
    # we assume df_sp500['date'] and 'close_price' exist and df_sp500 sorted
    end = df_sp500['date'].max()
    one_year = df_sp500[df_sp500['date'] >= end - pd.DateOffset(years=1)].copy().reset_index(drop=True)
    five_year = df_sp500[df_sp500['date'] >= end - pd.DateOffset(years=5)].copy().reset_index(drop=True)
    ten_year = df_sp500.copy().reset_index(drop=True)

    results = {}
    for label, df in [('1Y', one_year), ('5Y', five_year), ('10Y', ten_year)]:
        print(f"\n--- Running backtest for {label} ({len(df)} rows) ---")
        # Prepare df: ensure needed columns exist
        if 'close_price' not in df.columns:
            raise ValueError("df must contain 'close_price' column")
        df = compute_bollinger(df)
        res = backtest(df)
        results[label] = res
        plot_backtest_results(res, title_prefix=label)

    return results

# To run:
results = run_all_backtests(df_sp500)



--- Running backtest for 1Y (251 rows) ---


C:\Users\jehil\AppData\Local\Temp\ipykernel_31112\1084094320.py:194: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\jehil\AppData\Local\Temp\ipykernel_31112\1084094320.py:203: RuntimeWarning:

invalid value encountered in scalar power



=== Performance Metrics ===
initial_capital_estimate: 5956.060059
ending_portfolio_value: -516.094420
total_return: -1.086650
CAGR: nan
annual_volatility: 39.230656
sharpe: -0.260664
max_drawdown: -1.123923
num_trades: 3
win_rate: 0.666667
profit_factor: 1.295142


,entry_index,entry_date,entry_price,exit_index,exit_date,exit_price,size,pnl
0,61,2025-02-26,5956.060059,80,2025-03-25,5726.873999,1,-234.912934
1,88,2025-04-04,5074.080078,102,2025-04-25,5364.678003,1,285.233247
2,219,2025-10-13,6654.720215,225,2025-10-21,6680.413013,1,19.012385



--- Running backtest for 5Y (1256 rows) ---


...

,entry_index,entry_date,entry_price,exit_index,exit_date,exit_price,size,pnl
0,67,2021-03-05,3841.939941,71,2021-03-11,3879.839014,1,34.019233
1,115,2021-05-13,4112.500000,117,2021-05-17,4169.300562,1,52.631261
2,205,2021-09-21,4354.189941,223,2021-10-15,4380.894946,1,22.324110
3,253,2021-11-29,4655.270020,260,2021-12-08,4647.694507,1,-12.223207
4,289,2022-01-20,4482.729980,299,2022-02-03,4539.949438,1,52.679509
5,313,2022-02-24,4288.700195,328,2022-03-17,4300.521484,1,7.520768
6,354,2022-04-25,4296.120117,378,2022-05-27,4035.770508,1,-264.385380
7,389,2022-06-14,3735.479980,404,2022-07-07,3830.680017,1,91.369357
8,443,2022-08-31,3955.000000,451,2022-09-13,4081.165552,1,122.084386
9,460,2022-09-26,3655.040039,477,2022-10-19,3672.991492,1,14.278461



--- Running backtest for 10Y (2515 rows) ---


C:\Users\jehil\AppData\Local\Temp\ipykernel_31112\1084094320.py:194: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\jehil\AppData\Local\Temp\ipykernel_31112\1084094320.py:203: RuntimeWarning:

invalid value encountered in scalar power



=== Performance Metrics ===
initial_capital_estimate: 1943.089966
ending_portfolio_value: -4164.368172
total_return: -3.143168
CAGR: nan
annual_volatility: 138.602348
sharpe: -0.319696
max_drawdown: -2.861789
num_trades: 40
win_rate: 0.750000
profit_factor: 1.452447


,entry_index,entry_date,entry_price,exit_index,exit_date,exit_price,size,pnl
0,28,2016-01-07,1943.089966,44,2016-02-01,1919.636993,1,-25.372609
1,146,2016-06-27,2000.540039,150,2016-07-01,2082.242004,1,79.619723
2,199,2016-09-12,2159.040039,207,2016-09-22,2160.949011,1,-0.251977
3,236,2016-11-02,2097.939941,240,2016-11-08,2128.584985,1,28.516459
4,331,2017-03-22,2348.449951,338,2017-03-31,2363.795496,1,12.981749
5,348,2017-04-17,2349.010010,352,2017-04-21,2351.987488,1,0.625491
6,401,2017-06-30,2423.409912,409,2017-07-13,2432.363989,1,6.521713
7,430,2017-08-11,2441.320068,444,2017-08-31,2453.352527,1,9.579106
8,552,2018-02-06,2695.139893,565,2018-02-26,2721.980481,1,24.118608
9,584,2018-03-23,2588.260010,598,2018-04-13,2649.181494,1,58.272303


In [37]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

# ---------- PARAMETERS ----------
BB_WINDOW = 20
BB_STD_MULT = 2.0
position_size = 1
commission_pct = 0.001          # 0.1% per trade
risk_free_rate = 0.0
trading_days_per_year = 252

# ---------- UTILITIES ----------
def compute_bollinger(df, window=20, std_mult=2.0):
    df = df.copy()
    df['sma'] = df['close_price'].rolling(window).mean()
    df['std'] = df['close_price'].rolling(window).std()
    df['upper'] = df['sma'] + std_mult * df['std']
    df['lower'] = df['sma'] - std_mult * df['std']
    return df

def get_execution_price(df, idx):
    if 'open' in df.columns and not pd.isna(df.loc[idx, 'open']):
        return df.loc[idx, 'open']
    else:
        return df.loc[idx, 'close_price']

# ---------- STRATEGY C SIGNALS ----------
def generate_signals_trend(df):
    df = df.copy().reset_index(drop=True)
    
    # Trend breakout: price > upper band
    df['breakout'] = df['close_price'] > df['upper']
    
    # Trend weakening: price < SMA
    df['exit_cond'] = df['close_price'] < df['sma']
    
    df['signal'] = 0
    df.loc[df['breakout'], 'signal'] = 1    # entry
    df.loc[df['exit_cond'], 'signal'] = -1  # exit
    
    # shift so signals execute next bar
    df['exec_signal'] = df['signal'].shift(1).fillna(0).astype(int)
    return df

# ---------- BACKTEST ENGINE ----------
def backtest_trend(df):
    df = df.copy().reset_index(drop=True)
    df = compute_bollinger(df)
    df = generate_signals_trend(df)

    position = 0
    entry_price = np.nan
    cash = 0.0
    equity = []
    trades = []
    
    df['trade_price'] = np.nan
    df['trade_type'] = None
    df['position'] = 0
    df['equity'] = 0.0
    df['cash'] = 0.0
    df['pnl'] = 0.0

    for i in range(len(df)):
        sig = int(df.loc[i, 'exec_signal'])

        # ENTRY
        if sig == 1 and position == 0:
            price = get_execution_price(df, i)
            comm = commission_pct * price * position_size
            cash -= price * position_size + comm
            entry_price = price
            position = 1
            
            df.loc[i, 'trade_type'] = 'entry'
            df.loc[i, 'trade_price'] = price

            trades.append({
                'type': 'entry',
                'index': i,
                'date': df.loc[i, 'date'],
                'price': price,
                'commission': comm
            })

        # EXIT
        elif sig == -1 and position == 1:
            price = get_execution_price(df, i)
            comm = commission_pct * price * position_size
            
            gross_pnl = (price - entry_price) * position_size
            realized = gross_pnl - comm
            
            cash += price * position_size - comm
            position = 0
            
            df.loc[i, 'trade_type'] = 'exit'
            df.loc[i, 'trade_price'] = price
            df.loc[i, 'pnl'] = realized

            trades.append({
                'type': 'exit',
                'index': i,
                'date': df.loc[i, 'date'],
                'price': price,
                'commission': comm,
                'realized_pnl': realized,
                'entry_price': entry_price
            })
            entry_price = np.nan

        # Position tracking
        df.loc[i, 'position'] = position
        if position == 1:
            current_price = df.loc[i, 'close_price']
            unrealized = (current_price - entry_price) * position_size
        else:
            unrealized = 0

        equity_val = cash + unrealized
        df.loc[i, 'equity'] = equity_val
        df.loc[i, 'cash'] = cash
        equity.append(equity_val)

    # --------------------------
    # Aggregate trades into pairs
    paired = []
    entry = None
    for t in trades:
        if t['type'] == 'entry':
            entry = t
        elif t['type'] == 'exit' and entry is not None:
            paired.append({
                'entry_date': entry['date'],
                'entry_price': entry['price'],
                'exit_date': t['date'],
                'exit_price': t['price'],
                'pnl': t['realized_pnl']
            })
            entry = None

    trades_df = pd.DataFrame(paired)

    # --------------------------
    # Performance metrics
    pv = df['equity'].copy()
    if pv.iloc[0] <= 0:
        pv += abs(pv.iloc[0]) + 1
    daily = pv.pct_change().fillna(0)

    total_return = pv.iloc[-1] / pv.iloc[0] - 1
    years = (df['date'].iloc[-1] - df['date'].iloc[0]).days / 365.25
    CAGR = (pv.iloc[-1] / pv.iloc[0]) ** (1 / years) - 1 if years > 0 else np.nan
    ann_vol = daily.std() * np.sqrt(trading_days_per_year)
    sharpe = (daily.mean() * trading_days_per_year) / ann_vol if ann_vol > 0 else np.nan
    max_dd = (pv - pv.cummax()) / pv.cummax()
    max_dd = max_dd.min()

    win_rate = (trades_df['pnl'] > 0).mean() if len(trades_df) else np.nan
    profit_factor = (
        trades_df[trades_df['pnl'] > 0]['pnl'].sum() /
        abs(trades_df[trades_df['pnl'] <= 0]['pnl'].sum())
        if np.any(trades_df['pnl'] <= 0) else np.nan
    )

    metrics = {
        'total_return': total_return,
        'CAGR': CAGR,
        'sharpe': sharpe,
        'max_drawdown': max_dd,
        'num_trades': len(trades_df),
        'win_rate': win_rate,
        'profit_factor': profit_factor,
    }

    return {'df': df, 'trades': trades_df, 'metrics': metrics}

# ---------- PLOTTING ----------
def plot_backtest_trend(result, title="Trend BB Backtest"):
    df = result['df']
    trades = result['trades']

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        row_heights=[0.65, 0.35],
                        subplot_titles=(title, "Equity Curve"))

    fig.add_trace(go.Scatter(x=df['date'], y=df['close_price'], name="Close", mode='lines'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['date'], y=df['upper'], name="Upper", mode='lines'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['date'], y=df['sma'], name="SMA", mode='lines'), row=1, col=1)

    # markers
    fig.add_trace(go.Scatter(
        x=trades['entry_date'], y=trades['entry_price'],
        mode='markers', marker=dict(symbol='triangle-up', size=10),
        name="Entry"
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=trades['exit_date'], y=trades['exit_price'],
        mode='markers', marker=dict(symbol='triangle-down', size=10),
        name="Exit"
    ), row=1, col=1)

    fig.add_trace(go.Scatter(x=df['date'], y=df['equity'], name="Equity", mode='lines'), row=2, col=1)

    fig.update_layout(height=900)
    fig.show()

    print("\n=== PERFORMANCE METRICS ===")
    for k, v in result['metrics'].items():
        print(f"{k}: {v:.6f}" if isinstance(v, float) else f"{k}: {v}")

    print("\n=== TRADES ===")
    display(trades)

# ---------- RUN ON YOUR 1Y / 5Y / 10Y DATA ----------
def run_trend_backtests(df_sp500):
    end = df_sp500['date'].max()
    one_year = df_sp500[df_sp500['date'] >= end - pd.DateOffset(years=1)].copy().reset_index(drop=True)
    five_year = df_sp500[df_sp500['date'] >= end - pd.DateOffset(years=5)].copy().reset_index(drop=True)
    ten_year = df_sp500.copy().reset_index(drop=True)

    results = {}
    for label, df in [('1Y', one_year), ('5Y', five_year), ('10Y', ten_year)]:
        print(f"\n\n--- Running Trend Breakout Backtest: {label} ---")
        result = backtest_trend(df)
        plot_backtest_trend(result, title=f"{label} Bollinger Trend Strategy")
        results[label] = result

    return results

# RUN:
results_trend = run_trend_backtests(df_sp500)




--- Running Trend Breakout Backtest: 1Y ---



=== PERFORMANCE METRICS ===
total_return: 94.235417
CAGR: 94.533090
sharpe: -1.016639
max_drawdown: -6098.252316
num_trades: 4
win_rate: 0.500000
profit_factor: 1.687426

=== TRADES ===


,entry_date,entry_price,exit_date,exit_price,pnl
0,2025-06-25,6092.160156,2025-08-04,6329.939941,231.449845
1,2025-08-13,6466.580078,2025-08-22,6466.910156,-6.136832
2,2025-09-12,6584.290039,2025-10-13,6654.720215,63.775456
3,2025-10-28,6890.890137,2025-11-07,6728.799805,-168.819132




--- Running Trend Breakout Backtest: 5Y ---



=== PERFORMANCE METRICS ===
total_return: 455.843981
CAGR: 2.404289
sharpe: -0.365576
max_drawdown: -3704.350552
num_trades: 21
win_rate: 0.428571
profit_factor: 1.736026

=== TRADES ===


,entry_date,entry_price,exit_date,exit_price,pnl
0,2021-01-04,3700.649902,2021-01-28,3787.379883,82.942601
1,2021-04-06,4073.939941,2021-05-12,4063.040039,-14.962942
2,2021-07-02,4352.339844,2021-07-20,4323.060059,-33.602845
3,2021-08-31,4522.680176,2021-09-13,4468.729980,-58.418925
4,2021-10-20,4536.189941,2021-11-29,4655.270020,114.424808
5,2021-12-28,4786.350098,2022-01-06,4696.049805,-94.996343
6,2022-07-21,3998.949951,2022-08-23,4128.729980,125.651299
7,2022-10-26,3830.600098,2022-11-04,3770.550049,-63.820599
8,2022-11-14,3957.250000,2022-12-07,3933.919922,-27.263998
9,2023-01-17,3990.969971,2023-02-21,3997.340088,2.372777




--- Running Trend Breakout Backtest: 10Y ---



=== PERFORMANCE METRICS ===
total_return: 398.554640
CAGR: 0.820212
sharpe: -0.514617
max_drawdown: -2350.223819
num_trades: 40
win_rate: 0.425000
profit_factor: 1.510225

=== TRADES ===


,entry_date,entry_price,exit_date,exit_price,pnl
0,2016-04-14,2082.780029,2016-04-29,2065.300049,-19.545281
1,2016-05-26,2090.100098,2016-06-14,2075.320068,-16.855349
2,2016-12-08,2246.189941,2017-01-03,2257.830078,9.382307
3,2017-01-26,2296.679932,2017-03-15,2385.260010,86.194818
4,2017-04-26,2387.449951,2017-05-18,2365.719971,-24.095700
5,2017-06-05,2436.100098,2017-06-28,2440.689941,2.149154
6,2017-07-17,2459.139893,2017-08-11,2441.320068,-20.261144
7,2017-09-13,2498.370117,2017-11-16,2585.639893,84.684135
8,2017-11-29,2626.070068,2018-02-05,2648.939941,20.220933
9,2018-05-11,2727.719971,2018-05-30,2724.010010,-6.433971
